In [ ]:
import pandas as pd
import numpy as np
import altair as alt
from joblib import Parallel, delayed
from numpy.random import Generator, PCG64
rng = np.random.default_rng()

# Data Preprocessing

In [ ]:
aoa_df = pd.read_csv("../data/word_age_of_acquisition.csv")
aoa_df = aoa_df.replace({"comb (object)": "comb"})
df = pd.read_csv("../data/kisumu_vocab_data.csv")

In [ ]:
word_classification = pd.read_csv("../data/word_classification_df.csv")
word_classification['Target'] = word_classification['Target'].apply(lambda w : w.lower())
word_classification['Near Distractor'] = word_classification['Near Distractor'].apply(lambda w : w.lower())
word_classification['Random 1'] = word_classification['Random 1'].apply(lambda w : w.lower())
word_classification['Random 2'] = word_classification['Random 2'].apply(lambda w : w.lower())

In [ ]:
def find_word(r):
    target = r['target_word'].title()
    word_response = subject_responses[
        (subject_responses["Subject Number"] == r["child"])
    ].iloc[0][target]

    return word_response.lower()

def classify_response(r):
    res = r['response']
    if res == r['Target']:
        return 'Target'
    elif res == r['Near Distractor']:
        return 'Near Distractor'
    elif res == r['Random 1'] or r['Random 2']:
        return 'Random'
    else:
        return ''

subject_responses = pd.read_csv("../data/Kisumu_2024_Vocabulary_Vocab.csv").dropna(subset=["Subject Number"])
all_df = pd.merge(df, word_classification, left_on="target_word", right_on="Target", how="left")
all_df = pd.merge(all_df, aoa_df, on='target_word', how='left')
all_df['response'] = all_df.apply(find_word, axis=1)
all_df['response_type'] = all_df.apply(classify_response, axis=1)
all_df.to_csv("../data/all_df.csv")

# Error Plot

In [ ]:
filtered_df = all_df.copy()
filtered_df = filtered_df[filtered_df['accuracy'] == 0]
filtered_df['is_near_distractor'] = (filtered_df['response_type'] == 'Near Distractor').astype(int)
filtered_df['is_random'] = (filtered_df['response_type'] == 'Random').astype(int)
filtered_df['condition'] = filtered_df['condition'].replace({
    'bw': 'black_white',
    'obj': 'object',
})

In [ ]:
def bootstrap_ci(
        data,
        measure,
        id_col,
        n_iterations=10000,
        statistic=np.mean):
    
    items = list(data[id_col].unique())
    n_size = len(items)
    df = data.copy()

    def bootstrap_iteration(data, chosen_items):
        filter_df = data[data[id_col].isin(chosen_items)] # Filter based on chosen questions
        bs_mean = statistic(filter_df[measure]) 
        return (bs_mean, list(chosen_items))

    qset_means = Parallel(n_jobs=-1)(
        delayed(bootstrap_iteration)(
            df.copy(),
            rng.choice(items, n_size,  replace=True)
        ) for _ in range(n_iterations)
    )
    
    means = []
    qs_used = []
    means = [bs_mean for bs_mean, chosen_qs in qset_means]
 
    # 95% confidence interval
    lower = np.percentile(means, 2.5)
    upper = np.percentile(means, 97.5)
    
    return lower, upper


def create_confidence_interval_df(
    data,
    measure, 
    id_col,
    condition_col,
    statistic=np.mean
):
    data_list = []

    for condition in data[condition_col].unique():
        condition_data = data[data[condition_col] == condition]

        lower, upper = bootstrap_ci(condition_data, measure=measure, statistic=statistic, id_col=id_col)

        data_list.append({
            "condition": condition,
            "ci_upper": upper, 
            "ci_lower": lower,
        })

    return pd.DataFrame(data_list)


In [ ]:
def get_error_dfs(df, measure):
    agg_data = df.groupby(['condition']).agg(
        mean_prop=(measure, 'mean')
    ).reset_index()
    
    word_level_data = df.groupby(['condition', 'target_word']).agg(
        mean_prop=(measure, 'mean')
    ).reset_index()
    word_level_data['item_id'] = word_level_data['condition'] + word_level_data['target_word']
    
    ci_df = create_confidence_interval_df(
        data=df,
        measure=measure,
        id_col='target_word',
        condition_col='condition'
    )

    error_df = pd.merge(agg_data, ci_df, on=['condition'])

    return error_df, word_level_data

error_dfs, item_dfs = get_error_dfs(
    filtered_df, 
    measure='is_random'
)

In [ ]:
def create_error_plot(error_df, item_level_df):
    ci_plot = alt.Chart(error_df).mark_errorbar().encode(
        x=alt.X("condition:N", title=None),
        y=alt.Y("ci_upper", title="Proportion of Far Distractors Selected"),
        y2=alt.Y2("ci_lower"),
        strokeWidth=alt.value(2),
        color=alt.Color('condition').legend(None)
    )

    mean_points = alt.Chart(error_df).mark_point(filled=True, size=75, opacity=1).encode(
        x=alt.X('condition:N', scale=alt.Scale(domain=["black_white", 'cartoon', 'photo', 'object'])),
        y=alt.Y('mean_prop:Q', scale=alt.Scale(domain=[0,1])),
        color='condition:N'
    )
    
    scatter_plot = alt.Chart(item_level_df).mark_circle(size=16,opacity=0.5).encode(
        x=alt.X("condition:N", title=None),
        y=alt.Y("mean_prop:Q", scale=alt.Scale(domain=[0,1])),
        xOffset="jitter:Q",
        color=alt.Color('condition:N').legend(None),
    ).transform_calculate(
        jitter="sqrt(-2*log(random()))*cos(2*PI*random())" 
    )
    
    dashed_line = alt.Chart(error_df).mark_rule(strokeDash=[5, 10], color='black').encode(
        y=alt.datum(0.66),
        opacity=alt.value(0.5)
    )
    
    return mean_points + scatter_plot + ci_plot + dashed_line

plot = create_error_plot(error_dfs, item_dfs)
plot = plot.properties(width=150, title="Proportion of Incorrect Responses")
# plot.save("../figs/proportion_incorrect_plot.pdf")
plot

# Model Comparisons

In [ ]:
all_selection_df = all_df.copy()
all_selection_df = all_selection_df[all_selection_df['accuracy'] == 0]
all_selection_df['is_near_distractor'] = (all_selection_df['response_type'] == 'Near Distractor').astype(int)
all_selection_df['is_random'] = (all_selection_df['response_type'] == 'Random').astype(int)
all_selection_df = all_selection_df[['child', 'condition', 'age', 'target_word', 'accuracy', 'response_type', 'is_near_distractor', 'is_random']]
all_selection_df.to_csv("../data/error_trials_only.csv")
# all_selection_df.to_csv("../data/all_selection_df.csv")

## Loading R

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
install.packages("lme4")
library(lme4)

## Model Comparison

The below is implemented in the `picture-perception-error-analysis.Rmd` file

In [ ]:
%%R -i all_selection_df

base_model <- glmer(
    is_random ~ 1  + (condition | child) + (condition * age | target_word), 
    data=all_selection_df, 
    family = "binomial"
)

condition_model <- glmer(
    is_random ~ condition * age + (condition | child) + (condition * age | target_word), 
    data=all_selection_df, 
    family = "binomial"
)

anova(base_model, condition_model, test = "Chisq")

In [ ]:
%%R
base_model <- glmer(
    is_random ~ 1  + (condition | child), 
    data=all_selection_df, 
    family = "binomial"
)

condition_model <- glmer(
    is_random ~ condition + (condition | child), 
    data=all_selection_df, 
    family = "binomial"
)

anova(base_model, condition_model, test = "Chisq")